In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%sqlcmd tables

Name
ana__resultats_annee_udi
ana__resultats_communes
ana__resultats_last_udi
ana__resultats_udi
cog_communes
edc_communes
edc_prelevements
edc_resultats
int__lien_commune_cdreseau
laposte_communes


In [5]:
%%sql
SELECT DISTINCT categorie FROM mapping_categories

Running query in 'duckdb:///../../database/data.duckdb'

categorie
métabolite de pesticide
hydrocarbures
pcb
dioxine et furane
médicament
minéral
nitrite
pfas
phtalate
paramètre organoleptique


In [9]:
%%sql
SELECT * FROM mapping_categories
WHERE libminparametre IN ('1,4 dioxane', 'perchlorate')

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
14DAN,1580,"1,4 DIOXANE","1,4 dioxane",123-91-1,hydrocarbures


In [11]:
%%sql
SELECT DISTINCT 
    cdparametresiseeaux, limitequal_float , limitequal
FROM int__resultats_udi_communes
WHERE cdparametresiseeaux IN ('14DAN')

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,limitequal_float,limitequal
14DAN,None,None
